# daemon讲解

### 目录
- 基本概念
- 基本原理 
- PEP 4134 设计
- python-daemon库
- python-daemon实例
- 附录 A: 一个python daemon的实现
- 附录 B: 其他daemon的实现


## 基本概念 

### 进程

进程是程序的一个运行实例，也是执行程序的过程。同一个程序可以执行多次，每次都对应一个进程。在每次执行程序时，进程开辟独立内存空间、IO接口。操作系统的一个重要功能就是进程管理，其功能主要是为进程分配内存空间，管理进程的相关信息等等。

在Linux中，可用$ps命令来查询正在运行的进程。关于ps命令可参加：https://blog.csdn.net/freeking101/article/details/53444530

### fork() 

#### fork 处理过程

在类Unix系统中，操作系统提供了fork()系统调用，用于创建新的进程。与其他系统调用不同，fork()有如下特性：  
- 参数：无参数
- 返回值
    - 如果成功创建一个子进程，对于父进程来说返回子进程ID
    - 如果成功创建一个子进程，对于子进程来说返回值为0
    - 如果为-1表示创建失败（比如内存溢出等，无法创建子进程）
- 在fork()返回后，父、子进程开始从下一条指令开始执行。

有了fork调用，一个进程可创建子进程去处理新任务。例如，Apache服务器就是由父进程监听端口，每当有新http请求时，就fork出子进程来处理新的http请求。

In [1]:
# 实例： 说明fork的父、子进程都是从fork()函数的下一句开始，并行执行。 多层树型结构
import os
#print('Process %s start. ppid=%s'%(os.getpid(), os.getppid()))
pid = os.fork()
pid = os.fork()
pid = os.fork()
# how many time to print?  指数级增长：2**n 次
print('(%s, %s<-%s)'%(pid, os.getpid(), os.getppid()))

(22249, 21871<-21670)
(22250, 22246<-21871)
(0, 22248<-22245)
(22248, 22245<-21871)
(0, 22249<-21871)
(22256, 22247<-22245)
(0, 22250<-22246)
(0, 22256<-22247)


#### fork 调用关系

fork系统调用之后，父子进程将交替执行。根据父、子进程的退出顺序有如下情况：
1. 如父进程先退出，子进程还没退出，子进程的父进程将变为init进程。（注：任何一个进程都必须有父进程） 
2. 如子进程先退出，父进程还没退出，子进程必须等到父进程捕获到了子进程的退出状态才真正结束，否则这个时候子进程就成为僵进程。

当然，子进程退出会发送SIGCHLD信号给父进程，可以选择忽略或使用信号处理函数接收处理就可以避免僵尸进程。

#### fork 子进程资源
fork出的子进程继承了父进程下面这些属性：
- uid,gid,euid,egid
- 附加组id(sgid,supplementary group id) //sgid引入原因是有时候希望这个用户属于多个其他部门，这些其他部门的gid就是sgid
- 进程组id,会话id
- SUID标记和SGID标记
- 控制终端
- 当前工作目录/根目录
- 文件创建时的umask
- 文件描述符的文件标志(close-on-exec)
- 信号屏蔽和处理
- 存储映射
- 资源限制  

未被继承的属性有：
- pid不同
- 进程时间被清空
- 文件锁没有继承
- 未处理信号被清空

https://cloud.tencent.com/developer/article/1012442
https://blog.csdn.net/u013210620/article/details/78710532
https://www.cnblogs.com/qianchengprogram/p/6605675.html
https://www.jianshu.com/p/e3f3d49093ca
https://www.jb51.net/article/128316.htm
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000

#### 跨平台的多进程

Windows没有fork调用，因此fork无法在Windows运行。但是，Python提供了如下跨平台进程模块：
- multiprocessing 提供了Process类、Pool类，创建子进程
- subprocess 

multiprocessing 模块提供了一个 Process 类来创建进程对象，创建子进程时，需传入执行函数及参数。其常见函数包括：
1. start()方法启动子进程
2. join()方法等待P进程执行结束，在P子进程结束前，挂起父进程。通常用于进程间的同步。

下面的例子演示了启动一个子进程并等待其结束：

In [7]:
from multiprocessing import Process
import os

# 子进程要执行的代码
def run_proc(name):
    print('Run child process %s (%s)...' % (name, os.getpid()))

print('Parent process %s.' % os.getpid())
p = Process(target=run_proc, args=('HelloWorld',))
print('Child process will start.')
p.start()
# join：如果不使用join那么run_proc可能在父进程结束之后运行。 
p.join()
print('Child process end.')

Parent process 21871.
Child process will start.
Run child process HelloWorld (22434)...
Child process end.


#### Pool 进程池（待修改）

如果要启动大量的子进程，可以用进程池的方式批量创建子进程，也就是使用 multiprocessing.Pool 类。 

In [15]:
from multiprocessing import Pool
import os, random

def long_time_task(name):
    print('Run process %s (%s)...' % (name, os.getpid()))
    sl = random.random() * 2
    time.sleep(sl)
    print('process %s runs %0.2f seconds. ' % (name, sl))

if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    p = Pool(4)
    for i in range(5):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 21871.
Run process 2 (23384)...
Run process 0 (23382)...
Run process 1 (23383)...
Run process 3 (23385)...
process 2 runs 0.06 seconds. 
Waiting for all subprocesses done...
Run process 4 (23384)...
process 4 runs 0.06 seconds. 
process 0 runs 0.54 seconds. 
process 3 runs 1.05 seconds. 
process 1 runs 1.17 seconds. 
All subprocesses done.


代码解读：

对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了。

请注意输出，有四个task是立刻执行的，输出Run task x，而最后一个task 要等待前面某个task完成后才执行，这是因为在Pool初始化的时候设定为4。因此，最多同时执行4个进程。
- 如果改成：p = Pool(5) 就可以同时跑5个进程；
- 改为 p = Pool(3) 就可以同时跑5个进程；
- 如果不设定，p = Pool(), 那么Pool的默认大小是电脑的CPU核数，我的电脑是4. 



#### subporcess 子进程（待修改，无法使用）
https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001431927781401bb47ccf187b24c3b955157bb12c5882d000
很多时候，子进程并不是自身，而是一个外部进程。我们创建了子进程后，还需要控制子进程的输入和输出。

subprocess模块可以让我们非常方便地启动一个子进程，然后控制其输入和输出。

下面的例子演示了如何在Python代码中运行命令nslookup www.biadu.com， 这和命令行直接运行的效果是一样的：

In [21]:
import subprocess

print('$ nslookup www.baidu.com')
r = subprocess.call(['nslookup', 'www.baidu.com'])
print('Exit code:', r)

$ nslookup www.baidu.com
Exit code: 0


In [19]:
!nslookup www.baidu.com

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
www.baidu.com	canonical name = www.a.shifen.com.
Name:	www.a.shifen.com
Address: 119.75.216.20
Name:	www.a.shifen.com
Address: 119.75.213.61



In [25]:
import threading

# 创建全局ThreadLocal对象:
local_school = threading.local()

def process_student():
    # 获取当前线程关联的student:
    std = local_school.student
    print('Hello, %s (in %s)' % (std, threading.current_thread().name))

def process_thread(name):
    # 绑定ThreadLocal的student:
    local_school.student = name
    process_student()

t1 = threading.Thread(target= process_thread, args=('Alice',), name='Thread-A')
t2 = threading.Thread(target= process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Alice (in Thread-A)
Hello, Bob (in Thread-B)


### 多线程（multithread）

### fock 进程